## This is a note experimenting on the robustness to kernel width of LIME and BayLIME
22/07/2020 

In [ ]:
import sys
sys.path.append("..")# allow the notebook to find the parent folder
from sklearn.datasets import load_boston,load_breast_cancer
import sklearn.ensemble
import sklearn.linear_model
import sklearn.model_selection
import numpy as np
from sklearn.metrics import r2_score
np.random.seed(1)
import lime
import lime.lime_tabular
from lime import submodular_pick
from lime import calculate_posteriors
from scipy import stats
import random
import math
import csv


#load example dataset
boston = load_boston()
#cancer= load_breast_cancer()

#print a description of the variables
print(boston.DESCR)
#print(cancer.DESCR)

#train a regressor
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=1000)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(boston.data, boston.target, train_size=0.80, test_size=0.20)
rf.fit(train, labels_train);

#train a classifier
# rf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000)
# train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(cancer.data, cancer.target, train_size=0.80, test_size=0.20)
# rf.fit(train, labels_train);

In [ ]:
# generate an "explainer" object
categorical_features  = np.argwhere(np.array([len(set(boston.data[:,x])) for x in range(boston.data.shape[1])]) <= 10).flatten()



# Now you can play with different kernel_width to see how the resutls change...
explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=5)



#generate an explanation for testing..
instance = 13
#use rf.predict_proba for classfication 
exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='non_Bay',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' 'BayesianRidge_inf_prior_fit_alpha'

exp.show_in_notebook(show_table=True)
#fig = exp.as_pyplot_figure(label=1)


exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='Bay_info_prior',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' 'BayesianRidge_inf_prior_fit_alpha'


alpha_init=1
lambda_init=1
with open('.\posterior_configure.csv') as csv_file:
    csv_reader=csv.reader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 1:
            alpha_init=float(row[0])
            lambda_init=float(row[1])
        line_count=line_count+1
    
exp.show_in_notebook(show_table=True)

In [ ]:
# define the global lipschitz function
def glo_lip (kw1, kw2, f1, f2):
    f1=np.array(f1)
    f2=np.array(f2)
    return np.linalg.norm(f1-f2)/np.linalg.norm(kw1-kw2)
print(glo_lip(2,4,[0.1,0.2,0.4,0.1],[0.2,0.2,0.1,0.1]))

In [ ]:
i=1
kw1_s=np.array([])
kw2_s=np.array([])
lip_s=np.array([])
while i<=5000:
    kw1=random.uniform(0.01,5)
    kw1_s=np.append(kw1_s,kw1)
    kw2=random.uniform(0.01,5)
    kw2_s=np.append(kw2_s,kw2)
    
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=kw1)
    #generate an explanation for testing..
    instance = 13
    #use rf.predict_proba for classfication 
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='Bay_non_info_prior',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior'
    
    temp=exp.as_list()
    temp = sorted(temp, key=lambda a_entry: a_entry[0])
    temp=np.delete(temp,0,axis=1).flatten()
    f1=[float(i) for i in temp]
    
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=kw2)
    #generate an explanation for testing..
    instance = 13
    #use rf.predict_proba for classfication 
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='Bay_non_info_prior',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior'
    
    temp=exp.as_list()
    temp = sorted(temp, key=lambda a_entry: a_entry[0])
    temp=np.delete(temp,0,axis=1).flatten()
    f2=[float(i) for i in temp]
    
    lip=glo_lip(kw1, kw2, f1, f2)
    lip_s=np.append(lip_s,lip)
    
    i=i+1
    
print(stats.describe(lip_s))
data=np.asarray((kw1_s,kw2_s,lip_s)).T
np.savetxt("./figures/robust_kw/raw_data/3d_plot_Bay_non_info_prior.csv", data, delimiter=",")

In [ ]:

i=1
kw1_s=np.array([])
kw2_s=np.array([])
lip_s=np.array([])
while i<=5000:
    kw1=random.uniform(0.01,5)
    kw1_s=np.append(kw1_s,kw1)
    kw2=random.uniform(0.01,5)
    kw2_s=np.append(kw2_s,kw2)
    
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=kw1)
    #generate an explanation for testing..
    instance = 13
    #use rf.predict_proba for classfication 
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='non_Bay',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' BayesianRidge_inf_prior_fit_alpha
    
    temp=exp.as_list()
    temp = sorted(temp, key=lambda a_entry: a_entry[0])
    temp=np.delete(temp,0,axis=1).flatten()
    f1=[float(i) for i in temp]
    
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=kw2)
    #generate an explanation for testing..
    instance = 13
    #use rf.predict_proba for classfication 
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='non_Bay',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' BayesianRidge_inf_prior_fit_alpha
    
    temp=exp.as_list()
    temp = sorted(temp, key=lambda a_entry: a_entry[0])
    temp=np.delete(temp,0,axis=1).flatten()
    f2=[float(i) for i in temp]
    
    lip=glo_lip(kw1, kw2, f1, f2)
    lip_s=np.append(lip_s,lip)
    
    i=i+1
    
print(stats.describe(lip_s))
data=np.asarray((kw1_s,kw2_s,lip_s)).T
np.savetxt("./figures/robust_kw/raw_data/3d_plot_non_Bay.csv", data, delimiter=",")

In [ ]:

i=1
kw1_s=np.array([])
kw2_s=np.array([])
lip_s=np.array([])
while i<=5000:
    kw1=random.uniform(0.01,5)
    kw1_s=np.append(kw1_s,kw1)
    kw2=random.uniform(0.01,5)
    kw2_s=np.append(kw2_s,kw2)
    
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=kw1)
    #generate an explanation for testing..
    instance = 13
    #use rf.predict_proba for classfication 
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='Bay_info_prior',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' BayesianRidge_inf_prior_fit_alpha
    
    alpha_init=1
    lambda_init=1
    with open('.\posterior_configure.csv') as csv_file:
        csv_reader=csv.reader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 1:
                alpha_init=float(row[0])
                lambda_init=float(row[1])
            line_count=line_count+1
        
    temp=exp.as_list()
    temp = sorted(temp, key=lambda a_entry: a_entry[0])
    temp=np.delete(temp,0,axis=1).flatten()
    f1=[float(i) for i in temp]
    
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=kw2)
    #generate an explanation for testing..
    instance = 13
    #use rf.predict_proba for classfication 
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='Bay_info_prior',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' BayesianRidge_inf_prior_fit_alpha
    
    alpha_init=1
    lambda_init=1
    with open('.\posterior_configure.csv') as csv_file:
        csv_reader=csv.reader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 1:
                alpha_init=float(row[0])
                lambda_init=float(row[1])
            line_count=line_count+1
    
    
    temp=exp.as_list()
    temp = sorted(temp, key=lambda a_entry: a_entry[0])
    temp=np.delete(temp,0,axis=1).flatten()
    f2=[float(i) for i in temp]
    
    lip=glo_lip(kw1, kw2, f1, f2)
    lip_s=np.append(lip_s,lip)
    
    i=i+1
    
print(stats.describe(lip_s))
data=np.asarray((kw1_s,kw2_s,lip_s)).T
np.savetxt("./figures/robust_kw/raw_data/3d_plot_Bay_info_prior_a1_l200.csv", data, delimiter=",")

In [ ]:

i=1
kw1_s=np.array([])
kw2_s=np.array([])
lip_s=np.array([])
while i<=5000:
    kw1=random.uniform(0.01,5)
    kw1_s=np.append(kw1_s,kw1)
    kw2=random.uniform(0.01,5)
    kw2_s=np.append(kw2_s,kw2)
    
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=kw1)
    #generate an explanation for testing..
    instance = 13
    #use rf.predict_proba for classfication 
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='BayesianRidge_inf_prior_fit_alpha',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' BayesianRidge_inf_prior_fit_alpha
    
    alpha_init=1
    lambda_init=1
    with open('.\posterior_configure.csv') as csv_file:
        csv_reader=csv.reader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 1:
                alpha_init=float(row[0])
                lambda_init=float(row[1])
            line_count=line_count+1
        
    temp=exp.as_list()
    temp = sorted(temp, key=lambda a_entry: a_entry[0])
    temp=np.delete(temp,0,axis=1).flatten()
    f1=[float(i) for i in temp]
    
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False,kernel_width=kw2)
    #generate an explanation for testing..
    instance = 13
    #use rf.predict_proba for classfication 
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='BayesianRidge_inf_prior_fit_alpha',
                                 num_samples=1000,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' BayesianRidge_inf_prior_fit_alpha
    
    alpha_init=1
    lambda_init=1
    with open('.\posterior_configure.csv') as csv_file:
        csv_reader=csv.reader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 1:
                alpha_init=float(row[0])
                lambda_init=float(row[1])
            line_count=line_count+1
    
    
    temp=exp.as_list()
    temp = sorted(temp, key=lambda a_entry: a_entry[0])
    temp=np.delete(temp,0,axis=1).flatten()
    f2=[float(i) for i in temp]
    
    lip=glo_lip(kw1, kw2, f1, f2)
    lip_s=np.append(lip_s,lip)
    
    i=i+1
    
print(stats.describe(lip_s))
data=np.asarray((kw1_s,kw2_s,lip_s)).T
np.savetxt("./figures/robust_kw/raw_data/3d_plot_BayesianRidge_inf_prior_fit_alpha_l200.csv", data, delimiter=",")

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 

print(lip_s)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
Z=np.array([])
for i in np.asarray(lip_s):
    if i!=0:
        Z=np.append(Z,math.log(i))
    else:
        Z=np.append(Z,0)
print(Z)

pnt3d=ax.scatter(kw1_s, kw2_s, Z,c=lip_s,marker='^')

ax.set_xlabel('kw1')
ax.set_ylabel('kw2')
ax.set_zlabel('lip')
ax.set_zlim(-5, 8)
#ax.set_ylim(0, 10)
#ax.set_xlim(0, 10)
cbar=plt.colorbar(pnt3d)

plt.show()

In [ ]:
np.savetxt("./figures/robust_kw/raw_data/3d_plot_BayesianRidge_inf_prior_fit_alpha_l200.csv", data, delimiter=",")